In [1]:
import pandas as pd
import holoviews as hv
import numpy as np
import sqlite3
import hvplot
import hvplot.pandas
from holoviews import opts
from contextlib import closing
from itertools import combinations

import operator
from functools import reduce
from fiutils.plot import *

pd.options.plotting.backend = 'holoviews'

hv.extension('bokeh')
opts.defaults(
    opts.Scatter(
        width=500, height=300,
    ),
    opts.HeatMap(
        width=500, height=400,
    ),
)

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
script = '000-simple-template'
db_name = f'data/{script}.db'
with closing(sqlite3.connect(db_name)) as db:
    x = pd.read_sql_query("SELECT * FROM sqlite_master", db)
    display(x)
    table_name = x.iloc[-1].tbl_name
    df = pd.read_sql_query(f"SELECT * FROM '{table_name}'", db)
df

,type,name,tbl_name,rootpage,sql
0,table,tab_20230623_155332_383668,tab_20230623_155332_383668,2,CREATE TABLE 'tab_20230623_155332_383668'(glit...
1,table,tab_20230703_184544_607487,tab_20230703_184544_607487,43,CREATE TABLE 'tab_20230703_184544_607487'(glit...
2,table,tab_20230703_184857_876720,tab_20230703_184857_876720,95,CREATE TABLE 'tab_20230703_184857_876720'(glit...


,glitch_v,glitch_time_ns,glitch_delay_ns,scan,timestamp,verdict,iter_t
0,0.725328,399,1544,0,1.688403e+09,FAIL,13824
1,0.996394,767,1439,1,1.688403e+09,MUTE,10069
2,1.149966,1121,1890,2,1.688403e+09,MUTE,9123
3,0.322748,258,9647,3,1.688403e+09,FAIL,8486
4,0.602602,588,2368,4,1.688403e+09,FAIL,8563
...,...,...,...,...,...,...,...
171630,1.199031,979,1970,171630,1.688403e+09,MUTE,10141
171631,1.441652,751,4653,171631,1.688403e+09,MUTE,10367
171632,1.188548,536,334,171632,1.688403e+09,MUTE,10395
171633,0.815162,695,8073,171633,1.688403e+09,MUTE,10248


In [3]:
summary_by(df, 'verdict')

,count,percent
verdict,,
FAIL,92503,53.90
MUTE,76324,44.47
GLITCH,2808,1.64
Total,171635,100.00


In [4]:
numerical_cols = list(df.select_dtypes(['int', 'float']).columns) + ['verdict']
multi_scatter(df, numerical_cols, 'verdict', nsample=1000, ncols=3, size=10)

:Layout
   .NdOverlay.I     :NdOverlay   [verdict]
      :Scatter   [glitch_v]   (glitch_time_ns)
   .NdOverlay.II    :NdOverlay   [verdict]
      :Scatter   [glitch_v]   (glitch_delay_ns)
   .NdOverlay.III   :NdOverlay   [verdict]
      :Scatter   [glitch_v]   (scan)
   .NdOverlay.IV    :NdOverlay   [verdict]
      :Scatter   [glitch_v]   (timestamp)
   .NdOverlay.V     :NdOverlay   [verdict]
      :Scatter   [glitch_v]   (iter_t)
   .NdOverlay.VI    :NdOverlay   [verdict]
      :Scatter   [glitch_v]   (verdict)
   .NdOverlay.VII   :NdOverlay   [verdict]
      :Scatter   [glitch_time_ns]   (glitch_delay_ns)
   .NdOverlay.VIII  :NdOverlay   [verdict]
      :Scatter   [glitch_time_ns]   (scan)
   .NdOverlay.IX    :NdOverlay   [verdict]
      :Scatter   [glitch_time_ns]   (timestamp)
   .NdOverlay.X     :NdOverlay   [verdict]
      :Scatter   [glitch_time_ns]   (iter_t)
   .NdOverlay.XI    :NdOverlay   [verdict]
      :Scatter   [glitch_time_ns]   (verdict)
   .NdOverlay.XII   :NdOverlay   [verdict]
      :Scatter   [glitch_delay_ns]   (scan)
   .NdOverlay.XIII  :NdOverlay   [verdict]
      :Scatter   [glitch_delay_ns]   (timestamp)
   .NdOverlay.XIV   :NdOverlay   [verdict]
      :Scatter   [glitch_delay_ns]   (iter_t)
   .NdOverlay.XV    :NdOverlay   [verdict]
      :Scatter   [glitch_delay_ns]   (verdict)
   .NdOverlay.XVI   :NdOverlay   [verdict]
      :Scatter   [scan]   (timestamp)
   .NdOverlay.XVII  :NdOverlay   [verdict]
      :Scatter   [scan]   (iter_t)
   .NdOverlay.XVIII :NdOverlay   [verdict]
      :Scatter   [scan]   (verdict)
   .NdOverlay.XIX   :NdOverlay   [verdict]
      :Scatter   [timestamp]   (iter_t)
   .NdOverlay.XX    :NdOverlay   [verdict]
      :Scatter   [timestamp]   (verdict)
   .NdOverlay.XXI   :NdOverlay   [verdict]
      :Scatter   [iter_t]   (verdict)

In [5]:
dims = ['glitch_time_ns', 'glitch_delay_ns', 'glitch_v']
multi_heat2d(df[df.verdict.apply(lambda x: 'GLITCH' in x)], dims, bins=10, cmap='magma')

:Layout
   .HeatMap.I   :HeatMap   [columns,index]   (value)
   .HeatMap.II  :HeatMap   [columns,index]   (value)
   .HeatMap.III :HeatMap   [columns,index]   (value)